In [ ]:
!git clone https://github.com/buryne/capstone-project.git

Cloning into 'capstone-project'...
remote: Enumerating objects: 412, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 412 (delta 26), reused 11 (delta 11), pack-reused 340
Receiving objects: 100% (412/412), 1.08 MiB | 9.21 MiB/s, done.
Resolving deltas: 100% (118/118), done.


In [43]:
!wget https://raw.githubusercontent.com/buryne/capstone-project/app-dev/ML/Recommendation_Model_FIXED.h5

--2023-12-21 08:05:06--  https://raw.githubusercontent.com/buryne/capstone-project/app-dev/ML/Recommendation_Model_FIXED.h5
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 72344 (71K) [application/octet-stream]
Saving to: ‘Recommendation_Model_FIXED.h5.2’

Recommendation_Mode 100%[===================>]  70.65K  --.-KB/s    in 0.01s   

2023-12-21 08:05:06 (5.86 MB/s) - ‘Recommendation_Model_FIXED.h5.2’ saved [72344/72344]



In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from keras.models import Model
from keras.models import load_model
import requests
from io import StringIO


#URL raw file di Github
Github_url_1 = "https://raw.githubusercontent.com/buryne/capstone-project/app-dev/ML/dataset/Dataset_wisata.csv"

#Mendapatkan konten file CSV dari Github
response = requests.get(Github_url_1)
data = StringIO(response.text)

#Load dataset
wisata = pd.read_csv(data)
#URL raw file di Github
github_url_2 = "https://raw.githubusercontent.com/buryne/capstone-project/app-dev/ML/Recommendation_Model_FIXED.h5"

In [45]:
# load model
model = load_model('Recommendation_Model_FIXED.h5')
# summarize model.
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 64)                1344      
                                                                 
 dense_25 (Dense)            (None, 32)                2080      
                                                                 
 dense_26 (Dense)            (None, 1)                 33        
                                                                 
Total params: 3457 (13.50 KB)
Trainable params: 3457 (13.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [46]:
model = load_model('Recommendation_Model_FIXED.h5')

In [47]:
place_name = range(1,20)

In [48]:
data_wisata = np.array(list(set(place_name)))

In [49]:
city_name = "Jakarta"

In [50]:
city = np.array([city_name for i in range(len(data_wisata))])

In [51]:
city_label_mapping = {'Jakarta': 0, 'Yogyakarta': 1, 'Bandung': 2, 'Semarang': 3, 'Surabaya': 4}

In [52]:
city_encoded = np.array([city_label_mapping[city_name] for city_name in city])

In [53]:
input_data = np.column_stack((city_encoded, data_wisata))

In [54]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

#Replace NaN with an empty string
wisata['Place_Name'] = wisata['Place_Name'].fillna('')

# Concatenate 'Place_Name', 'City', 'Category', and 'Rating' into a single text column
wisata['Combined_Info'] = wisata['Place_Name'] + ' ' + wisata['City']

# Menggunakan TfIdfVectorizer untuk mengonversi deskripsi menjadi vektor fitur
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(wisata['Combined_Info'])

# Menghitung skor kesamaan kosinus antar tempat wisata
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

# Construct the required TF-IDF matrix by fitting and transforming the data
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(wisata['Combined_Info'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(437, 706)

In [56]:
# Mendapatkan rekomendasi menggunakan model yang telah diload
def get_recommendations_with_load_model(model, city='City', num_recommendations=10):
    # Filter data for the specified city
    city_data = wisata[wisata['City'] == city]

    # If no places found in the specified city, return an empty DataFrame
    if city_data.empty:
        return pd.DataFrame(columns=['Place_Name', 'City'])

    # Pilih acak tempat wisata dari kota yang ditentukan
    random_place_index = np.random.choice(city_data.index)
    random_place_features = city_data.loc[random_place_index, 'Place_Name']

    # Transform the random place features using the TF-IDF vectorizer
    random_place_tfidf = tfidf_vectorizer.transform([random_place_features])

    # Calculate cosine similarity between the random place and all tourist places in the same city
    cosine_similarities = linear_kernel(random_place_tfidf, tfidf_matrix[city_data.index, :]).flatten()

    # Get indices of similar tourist places based on cosine similarity scores
    similar_places_indices = cosine_similarities.argsort()[:-num_recommendations-1:-1]

    # Return the most similar destinations (Place_Name and City)
    return city_data[['Place_Id', 'Place_Name','Description', 'Category', 'City', 'Price', 'Rating']].iloc[similar_places_indices]

# Mendapatkan rekomendasi dengan menggunakan model yang telah diload
recommendations_by_city_loaded_model = get_recommendations_with_load_model(load_model, city_name)
print(f"Top {len(recommendations_by_city_loaded_model)} Rekomendasi wisata: ")
print(recommendations_by_city_loaded_model)

Top 10 Rekomendasi wisata: 
    Place_Id                   Place_Name  \
40        41        Museum Bahari Jakarta   
23        24              Museum Nasional   
72        73              Museum Perangko   
16        17            Museum Fatahillah   
20        21                Museum Wayang   
59        60               Museum Tekstil   
19        20        Museum Taman Prasasti   
17        18        Museum Bank Indonesia   
73        74          Museum Tengah Kebun   
62        63  Museum Kebangkitan Nasional   

                                          Description Category     City  \
40  Museum Bahari adalah museum yang menyimpan kol...   Budaya  Jakarta   
23  Museum Nasional Republik Indonesia atau Museum...   Budaya  Jakarta   
72  Museum perangko dibangun dengan bentuk banguna...   Budaya  Jakarta   
16  Museum Fatahillah memiliki nama resmi Museum S...   Budaya  Jakarta   
20  Museum Wayang adalah sebuah museum yang berlok...   Budaya  Jakarta   
59  Museum cinta Tekstil m